In [7]:
class episode:
    
    def __init__(self,player,dol_state,cnt):
        self.player = player
        self.dol_state = dol_state
        self.cnt = cnt
        
    def __repr__(self):
        return "%3r번째 수 player : %r 착수 : %r"%(self.cnt,self.player,self.cnt)
    
    
class game:
    """
    게임을 시작한다.
    game_rule
    game_board
    player를 받고 게임을 시작한다.
    """
    def __init__(self,game_rule,game_board):
        self._game_rule = game_rule
        self._game_board = game_board
        self.cnt = 0
        self.history = []
        
    def play(self):
        while True:
            player = self._game_rule.whos_turn(self.cnt)
            try:
                dol_state = player.take_turn()
            except :
                print("?? 제대로 입력할래 ??")
                continue
            
            try:
                self._game_board.board = dol_state
            except wrongposition_Error as e:
                print(e)
                continue
            except giveup_Interrupt as e:
                print(e)
                print("기권한 ",player,"의 패배 !!")
                break
            else:
                self.cnt += 1
            
            #print result
            self.show_game()
            # 착수된 수들을 번호와 함께 히스토리에 저장.
            self.history.append(episode(player,dol_state,self.cnt))
            
            winner = self._game_rule.check_winner(self._game_board.board,dol_state)
            if winner:
                print("!! winner is ",player)
                break
            if self._game_rule.check_endgame(self._game_board.board):
                print("더 이상 착수가 불가능 합니다. 게임 종료")
                break
                    
    def show_game(self):
        print("*"*20)
        print("착수 회수 :",self.cnt)
        print(self._game_board)
        print("*"*20)
        
    def show_history(self):
        print("*"*30)
        print("*"*30)
        print("*"*11,"history","*"*10)
        for history in self.history:
            print(history)
        

In [8]:
class game_rule_yukmok:
    """
    6목 게임의 룰을 정의한다.
    1. 한번에 두 알씩 
    2. 차례는 번갈아가면서
    3. 시작은 흑돌
    4. 같은 색의 돌이 6개가 이어지면 끝
    5. 더 이상 둘 곳이 없으면 끝
    6. 게임 참여자는 두명
    7. 착수는 주어진 게임 판 안에서만 가능
    8. 흑돌 1
    9. 흰돌 2
    """
    def __init__(self,player1,player2):
        self._player1 = player1
        self._player2 = player2
        self.__player_setting()
        self.cnt = 0
    
        
    def __player_setting(self):
        # 플레이어 흑돌 흰돌 설정
        self._player1.dol = 1
        self._player2.dol = 2
        
    def whos_turn(self,cnt):
        if cnt%4 in [0,1]:
            return self._player1
        else:
            return self._player2
    
    ### chekcing function 
    def check_winner(self,board,dol_state):
        """
        착수된 위치에서만 6목이 성립하지는 찾으면 된다.
        7목은 우승에서 제외
        """
        ans = False
        dol,x,y = dol_state
        if self.check_horizontal(board,dol,x,y):
            ans = True
        elif self.check_vertical(board,dol,x,y):
            ans = True
        elif self.check_diagonal(board,dol,x,y):
            ans = True

        return ans
    
    def check_horizontal(self,board,dol,x,y):
        cnt = 1
        #check rightside
        for i in range(1,6):
            target_y = i+y
            target_x = x
            if self.target_check(board,dol,target_x,target_y):
                cnt+=1
            else:
                break
        for i in range(-1,-6,-1):    
            target_y = i+y
            target_x = x
            if self.target_check(board,dol,target_x,target_y):
                cnt+=1
            else:
                break
                
        if cnt == 6: return True
        else: return False
            
    def check_vertical(self,board,dol,x,y):
        cnt = 1
        #check up
        for i in range(1,6):
            target_x = i+x
            target_y = y
            if self.target_check(board,dol,target_x,target_y):
                cnt+=1
            else:
                break
        #check down
        for i in range(-1,-6,-1):    
            target_x = i+x
            target_y = y
            if self.target_check(board,dol,target_x,target_y):
                cnt+=1
            else:
                break
        if cnt == 6: return True
        else: return False
        
    def check_diagonal(self,board,dol,x,y):
        cnt = 1
        #check left_diagonal
        for i in range(1,6):
            target_x = i-x
            target_y = i+y
            if board[target_x-1][target_y-1] == dol:
                cnt+=1
            else:
                break
        for i in range(-1,-6,-1):    
            target_x = i-x
            target_y = i+y
            if board[target_x-1][target_y-1] == dol:
                cnt+=1
            else:
                break
        if cnt == 6:return True
        
        
        #check right_diagonal
        cnt = 1
        for i in range(1,6):
            target_x = i+x
            target_y = i+y
            if self.target_check(board,dol,target_x,target_y):
                cnt+=1
            else:
                break
        for i in range(-1,-6,-1):    
            target_y = i+y
            target_x = i+x
            if self.target_check(board,dol,target_x,target_y) :
                cnt+=1
            else:
                break
        if cnt == 6: return True
        else: return False
    
    def target_check(self,board,dol,x,y):
        ans = False
        try :
            if board[x-1][y-1] == dol:
                ans = True
        except IndexError as e:
            pass 
        return ans
        
        
    
    def check_endgame(self,board):
        for row in board:
            if 0 in row:
                return False
                


In [9]:
class yukmok_Exception_handler(Exception):
    def __init__(self):
        pass
class wrongposition_Error(yukmok_Exception_handler):
    def __init__(self,msg):
        self.message = msg
class giveup_Interrupt(yukmok_Exception_handler):
    def __init__(self,msg):
        self.message = msg

In [10]:
class game_board:
    """
    게임 판을 만든다.
    플레이어가 착수를 하는 판.
    게임의 진행 상태가 게임 보드에 저장된다.
    정사각형이다.
    돌이 없으면 0
    """
    # 판 세팅
    def __init__(self):
        self._board = [[ 0 for x in range(1,20)] for y in range(1,20)]
    
    @property
    def board(self):
        return self._board
    
    @board.setter
    def board(self,state):
        dol,x,y = state
        if x == -1 and y == -1:
            raise giveup_Interrupt("기권이군 게임을 종료합니다.")
        self.check_invalid(x,y)
        self.board[x-1][y-1] = dol
    
    def check_invalid(self,x,y):
        try:
            self._board[x-1][y-1]
        except IndexError :
            raise wrongposition_Error("판을 넘어갔어 다시")
        else:
            if self._board[x-1][y-1] != 0:
                raise wrongposition_Error("이미 착수된 자리다 다른 곳에 두자")
    
    
    def __repr__(self):
        board_state =""
        for row in self._board:
            for x in row:
                board_state += str(x)
                board_state += " "
            board_state+="\n"
            
        return board_state

In [11]:
class player:
    """
    player
    사용자의 행동, 사용자 정보를 담고 있다.
    """
    def __init__(self,name = "mac"):
        self.name = name
        self.dol = 0
    
    def take_turn(self):
        print()
        print(self.name,"의 차례입니다. (기권 : -1 -1)")
        x,y = [int(i) for i in input("돌을 놓을 위치 좌표 [예 : 1 1 ]: ").split()]
        return (self.dol,x,y)

    def __repr__(self):
        return "( 이름 : %r 돌 : %r )"%(self.name,self.dol)


In [13]:
if __name__ == "__main__":
    player1 = player(name = "yo")
    player2 = player(name = "woong")
    game_rule = game_rule_yukmok(player1,player2)
    board = game_board()
    game1 = game(game_rule,board)

    game1.play()
    game1.show_history()


yo 의 차례입니다. (기권 : -1 -1)
돌을 놓을 위치 좌표 [예 : 1 1 ]: 1 1
********************
착수 회수 : 1
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

********************

yo 의 차례입니다. (기권 : -1 -1)
돌을 놓을 위치 좌표 [예 : 1 1 ]: -1 -1
기권이군 게임을 종료합니다.
기권한  ( 이름 : 'yo' 돌 : 1 ) 의 패배 !!
******************************
****************